The (adapted) code in this notebook comes from: https://github.com/siddharthkale97/CartPole-v1-DeepQ-AI/blob/master/DeepQcartpole.py

In [6]:
import gym
import csv
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from statistics import mean
import h5py

In [7]:
LEARNING_RATE = 1e-3
MAX_MEMORY = 1000000
BATCH_SIZE = 20
GAMMA = 0.95
EXPLORATION_DECAY = 0.995
EXPLORATION_MIN = 0.01

In [8]:
class ScoreEvaluator:

    def __init__(self, max_len, average_of_last_runs, model = None):
        self.max_len = max_len
        self.score_table = deque(maxlen=self.max_len)
        self.model = model
        self.average_of_last_runs = average_of_last_runs

    def store_score(self, episode, step):
        self.score_table.append([episode, step])

    def evaluation_score(self, title = "Training"):
        return self.score_table

In [9]:
class Network:

    def __init__(self, observation_space, action_space):

        self.action_space = action_space
        self.memory = deque(maxlen=MAX_MEMORY)
        self.exploration_rate = 1.0

        self.model = Sequential()
        self.model.add(Dense(32, input_shape=(observation_space,), activation='relu'))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(self.action_space, activation='linear'))
        self.model.compile(loss='mse', optimizer=Adam(lr=LEARNING_RATE))

    def add_to_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def take_action(self, state):
        if np.random.rand() < self.exploration_rate:
            a = random.randrange(0, self.action_space)
            return a, True
        q_values = self.model.predict(state)
        return np.argmax(q_values[0]), False

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        else:
            minibatch = random.sample(self.memory, BATCH_SIZE)
            for state, action, reward, state_next, done in minibatch:
                Q = reward
                if not done:
                    Q = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
                Q_values = self.model.predict(state)
                Q_values[0][action] = Q
                self.model.fit(state, Q_values, verbose=0)
            self.exploration_rate *= EXPLORATION_DECAY
            self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

    def get_model(self):
        return self.model


In [10]:
class TrainSolver:

    def __init__(self, max_episodes):
        self.max_episodes = max_episodes
        self.score_table = deque(maxlen=400)
        self.average_of_last_runs = None
        self.model = None
        self.play_episodes = 100
        env = gym.make('CartPole-v1')
        observation_space = env.observation_space.shape[0]
        action_space = env.action_space.n
        self.solver = Network(observation_space, action_space)

    def train(self):
        env = gym.make('CartPole-v1')
        observation_space = env.observation_space.shape[0]
        action_space = env.action_space.n

        print("---------------------------------")
        print("Solver starts")
        print("---------------------------------")

        self.model = self.solver.get_model()
        episode = 0
        score_eval = ScoreEvaluator(400, 50, self.model)
        with open("Cartpole_DQN_extended_train_scores.csv", "w") as csvfile:
            header = ["episode", "epsilon", "score"]
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(header)
            
            n = 0
            action = None
            while episode < self.max_episodes:
                episode += 1
                state = env.reset()
                state = np.reshape(state, [1, observation_space])
                step = 0
                while True:
                    step += 1
                    if n == 0:
                        action, n_yes_or_no = self.solver.take_action(state)
                        if n_yes_or_no:
                            n = np.random.randint(0, 50)
                    else:
                        action = action
                        n -= 1
                    state_next, reward, done, info = env.step(action)
                    if not done:
                        reward = reward
                    else:
                        reward = -reward
                    state_next = np.reshape(state_next, [1, observation_space])
                    self.solver.add_to_memory(state, action, reward, state_next, done)
                    state = state_next

                    if done:
                        print("Run: " + str(episode) + ", exploration: " + str(self.solver.exploration_rate) + ", score: " + str(step))
                        writer.writerow([episode, self.solver.exploration_rate, step])
                        self.score_table.append([episode, step])
                        score_eval.store_score(episode, step)
                        break
                    self.solver.experience_replay()

        train_scores = score_eval.evaluation_score("100 trains")
    
    def return_trained_model(self):
        return self.model
    
    def play(self, play_episodes=100, load_model=False, model_weights_dir=None, trained_model=None):

        self.play_episodes = play_episodes
        if load_model is not False:
            if model_weights_dir is None:
                print("Can't load specified model")
            elif trained_model is None:
                print("Please pass a valid model as a parameter")
            else:
                model = trained_model
                model.load(model_weights_dir)
        else:
            model = self.model

        env = gym.make('CartPole-v1')
        observation_space = env.observation_space.shape[0]
        action_space = env.action_space.n
        episode = 0
        score_eval = ScoreEvaluator(400, 100, model)
        while episode < self.play_episodes:

            episode += 1
            state = env.reset()
            state = np.reshape(state, [1, observation_space])
            step = 0
            while True:
                step += 1
                action, ignore_bool = self.solver.take_action(state)
                state_next, reward, done, info = env.step(action)

                if not done:
                    reward = reward
                else:
                    reward = -reward
                state_next = np.reshape(state_next, [1, observation_space])
                self.solver.add_to_memory(state, action, reward, state_next, done)
                state = state_next

                if done:
                    print("Run: " + str(episode) + ", score: " + str(
                        step))
                    self.score_table.append([episode, step])
                    score_eval.store_score(episode, step)
                    break
                self.solver.experience_replay()
        
        play_scores = score_eval.evaluation_score("100 Plays")
        with open("Cartpole_DQN_extended_play_scores.csv", "w") as csvfile:
            header = ["episode", "score"]
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(header)
            for i in play_scores:
                i = tuple(i)
                episode, score = i[0], i[1]
                writer.writerow([episode, score])
        
    def save_model(self):
        self.model.save('cartpole_dqn_extended_model.h5')


if __name__ == "__main__":
    trainer = TrainSolver(100)
    trainer.train()
    trainer.play(100)
    trainer.save_model()

---------------------------------
Solver starts
---------------------------------
Run: 1, exploration: 1.0, score: 9
Run: 2, exploration: 1.0, score: 9
Run: 3, exploration: 0.9655206468094844, score: 9
Run: 4, exploration: 0.918316468354365, score: 11
Run: 5, exploration: 0.8778091417340573, score: 10
Run: 6, exploration: 0.8433051360508336, score: 9
Run: 7, exploration: 0.810157377815473, score: 9
Run: 8, exploration: 0.7744209942832988, score: 10
Run: 9, exploration: 0.7439808620067382, score: 9
Run: 10, exploration: 0.7076077347272662, score: 11
Run: 11, exploration: 0.6763948591909945, score: 10
Run: 12, exploration: 0.5997278763867329, score: 25
Run: 13, exploration: 0.5732736268885887, score: 10
Run: 14, exploration: 0.547986285490042, score: 10
Run: 15, exploration: 0.5238143793828016, score: 10
Run: 16, exploration: 0.500708706245853, score: 10
Run: 17, exploration: 0.47862223409330756, score: 10
Run: 18, exploration: 0.4598090507939749, score: 9
Run: 19, exploration: 0.4395266